<a href="https://colab.research.google.com/github/Decoding-Data-Science/airesidency/blob/main/weather_assistant4thoct.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openai==0.28 requests

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 1.2 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.109.1
    Uninstalling openai-1.109.1:
      Successfully uninstalled openai-1.109.1


In [ ]:
!pip install gradio

In [ ]:
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get('openai')
#os.environ["weather_api_key"] = userdata.get('weatherai')

In [ ]:
import requests

def get_current_weather(location, unit='celsius'):
    weather_api_key = "a7b109315b6ced6a9cd5177bdb98ca82"
    base_url = f"http://api.openweathermap.org/data/2.5/weather?q={location}&appid={weather_api_key}&units=metric"
    response = requests.get(base_url)
    data = response.json()

    weather_description = data['weather'][0]['description']
    return {
        "location": location,
        "temperature": data['main']['temp'],
        "weather": weather_description
    }

In [ ]:
print(get_current_weather("Dubai"))

{'location': 'Dubai', 'temperature': 35.96, 'weather': 'clear sky'}


In [ ]:
functions = [
    {
        "name": "get_current_weather",
        "description": "Get the current weather in a given location",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city, e.g. San Francisco"
                },
                "unit": {
                    "type": "string",
                    "enum": ["celsius", "fahrenheit"]
                }
            },
            "required": ["location"]
        }
    }
]

In [ ]:
functions

[{'name': 'get_current_weather',
  'description': 'Get the current weather in a given location',
  'parameters': {'type': 'object',
   'properties': {'location': {'type': 'string',
     'description': 'The city, e.g. San Francisco'},
    'unit': {'type': 'string', 'enum': ['celsius', 'fahrenheit']}},
   'required': ['location']}}]

In [ ]:
import openai

#Function Definition and Initial Message Handling


def weather_chat(user_message):
    messages=[]
    messages.append({"role": "user", "content": user_message})
    messages.append({"role": "assistant", "content": "You are a weather bot . Answer only in Celsius. answer only weather related questions and politely decline if anything else"})



        # Sending Initial Message to OpenAI
    response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            temperature = 0,
            max_tokens=256,
            top_p=0.2,
            frequency_penalty=0,
            presence_penalty=0,
            messages=messages,
            functions=functions
        )

    #Handling Function Calls and Fetching Weather Data
    try:
        function_call = response['choices'][0]['message']['function_call']
        arguments = eval(function_call['arguments'])

        # Fetch weather data using the extracted arguments
        weather_data = get_current_weather(arguments['location'])

        # Append the function call and weather data to the messages
        messages.append({"role": "assistant", "content": None, "function_call": {"name": "get_current_weather", "arguments": str(arguments)}})
        messages.append({"role": "function", "name": "get_current_weather", "content": str(weather_data)})

#magic of llm
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=messages
        )

        return response['choices'][0]['message']['content']
    except Exception as e:
        return "I'm here to provide weather updates. Please ask me questions related to weather."



1.	try...except block: This is used to gracefully handle potential errors that might occur during the function calling process.
2.	function_call = response['choices'][0]['message']['function_call']: This line extracts the function call information from the OpenAI model's response.
3.	arguments = eval(function_call['arguments']): The arguments for the function call are extracted and evaluated.
4.	weather_data = get_current_weather(arguments['location']): Your get_current_weather function is called with the extracted location to fetch the weather data.
5.	messages.append(...): The function call and the resulting weather data are added back to the messages list. This is crucial for the model to understand the context and generate a human-readable response based on the weather data.
6.	response = openai.ChatCompletion.create(...): A second call is made to the OpenAI model, this time with the function call and its result included in the conversation history. This allows the model to synthesize the weather data into a natural language response.
7.	return response['choices'][0]['message']['content']: The final, human-readable weather update from the model is returned.
8.	except Exception as e:: If any error occurs during this process, a polite message is returned to the user indicating that the bot is focused on weather updates.


In [ ]:
weather_chat("is it hot in dubai now")
#i want to go out in the sun in dubai now is it viable

'Currently, the temperature in Dubai is 35.96°C with clear skies.'

In [ ]:
#qc
weather_chat("كم درجة الحرارة في الشارقة؟ ")

'درجة الحرارة في الشارقة الآن هي تقريباً 35.16 درجة مئوية مع سماء صافية.'

In [ ]:
import gradio as gr
# Define Gradio interface
iface = gr.Interface(
    fn=weather_chat,
    inputs=gr.Textbox(label="Weather Queries"),
    outputs=gr.Textbox(label="Weather Updates"),
    title = "DDS Weather Bot",
    description = "Ask me anything about weather!"
)

# Launch the Gradio interface
iface.launch(share="True")

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://9691551a69b2632127.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
